In [1]:
%pip install tensorflow_datasets
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import KFold

2024-05-13 15:11:11.026489: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 15:11:11.029370: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 15:11:11.067612: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 15:11:11.997704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "../../dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=32, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "../../models/data_with_augmentasi/training_batchSize32_epoch100.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.3f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-13 15:11:16.564065: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  22  24  25  26  27  28  29  32  34  35  36  37  38  40  41  42  43
  44  45  46  47  48  50  51  52  53  56  57  58  59  60  61  62  64  68
  69  70  71  73  74  75  80  82  83  85  87  88  89  90  91  92  93  94
  95  96  98  99 100 102 103 104 105 106 107 108 110 111 112 113 114 115
 116 117 119 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136
 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 156 157
 158 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177
 178 179 180 182 183 184 185 186 187 188 189 190 191 194 195 197 198 200
 201 202 203 205 206 207 214 216 217 219 221 222 223 224 225 226 228 229
 230 231 232 233 235 236 237 238 240 241 242 243 245 246 247 248 249 251
 252 253 255 256 257 258 260 261 262 263 264 266 267 268 269 270 271 272
 273 274 276 277 278 279 280 282 283 284 285 286 287 288 289 291 292 293
 294 295 297 298 30

Predicted Classes: [40 28  3 38 58 20 46 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 28 21 15 52 41  3 18 37 13  0 43 46 16  6  6 25 44 30  3
  0 27 58  5 17 39 53 57 13 32 18 42 37 38 50 25 32 57 18 43 40 16 38 53
 55 40 46 51 56  1 48  7 44  0 21 13 40 39 27 47 11 33 38 54 37 23 18  0]
True Classes: [40 28  3 38 58 20 46 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 2

Penambahan DropOut

In [4]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "../../dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Adding dropout layer with 50% dropout rate
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=32, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "../../models/data_with_augmentasi/training_withDropOut_batchSize32_epoch100.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.3f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-13 15:42:35.673669: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  22  24  25  26  27  28  29  32  34  35  36  37  38  40  41  42  43
  44  45  46  47  48  50  51  52  53  56  57  58  59  60  61  62  64  68
  69  70  71  73  74  75  80  82  83  85  87  88  89  90  91  92  93  94
  95  96  98  99 100 102 103 104 105 106 107 108 110 111 112 113 114 115
 116 117 119 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136
 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 156 157
 158 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177
 178 179 180 182 183 184 185 186 187 188 189 190 191 194 195 197 198 200
 201 202 203 205 206 207 214 216 217 219 221 222 223 224 225 226 228 229
 230 231 232 233 235 236 237 238 240 241 242 243 245 246 247 248 249 251
 252 253 255 256 257 258 260 261 262 263 264 266 267 268 269 270 271 272
 273 274 276 277 278 279 280 282 283 284 285 286 287 288 289 291 292 293
 294 295 297 298 30

Predicted Classes: [40 28  3 38 58 20 46 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 37 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 28 21 15 52 41  3 18 37 13  0 43 46 16  6  6 25 44 30  3
  0 27 58  5 17 39 53 57 13 32 18 42 37 38 50 25 32 57 18 43 40 16 38 53
 55 40 46 51 56  1 48  7 44  0 21 13 40 39 27 47 11 33 38 54 37 23 18  0]
True Classes: [40 28  3 38 58 20 46 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 2